# Definitions

In [1]:
import requests
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

In [2]:
sequences = pd.read_csv('../datasets/sequences.csv')
sequences.sample()

,Accession,SRA_Accession,Release_Date,Species,Genus,Family,Length,Sequence_Type,Nuc_Completeness,Genotype,...,Authors,Publications,Protein,Geo_Location,USA,Host,Isolation_Source,Collection_Date,BioSample,GenBank_Title
3213,QOT58300,SRR12895773,2020-10-28T00:00:00Z,Severe acute respiratory syndrome-related coro...,Betacoronavirus,Coronaviridae,1273,GenBank,complete,NaN,...,"Seemann,T., Caly,L., Sait,M., Schultz,M.B., Dr...",NaN,surface glycoprotein,Australia: Victoria,NaN,Homo sapiens,NaN,2020-08-05,SAMN16548891,surface glycoprotein [Severe acute respiratory...


# Data processing

In [3]:
sequences_extended = sequences.set_index('Accession')
sequences_extended['Sequence'] = ""
for accessions in tqdm(np.array_split(sequences_extended.index, 100)):
    response = requests.post(f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&rettype=fasta&retmode=text', {'id': ",".join(accessions)})
    sequences_extended.loc[accessions, 'Sequence'] = [s.strip() for s in re.split(r'>.*', response.text)[1:]]
    sequences_extended.to_csv('../datasets/sequences_extended.csv')

100%|██████████| 100/100 [08:33<00:00,  5.14s/it]


In [8]:
sequences_extended_filtered = sequences_extended[sequences_extended['Sequence'].apply(lambda s: 'X' not in s)][sequences_extended['Family'] == "Coronaviridae"].sort_values('Collection_Date').drop_duplicates(subset=['Sequence'])
sequences_extended_filtered.to_csv('../datasets/sequences_extended_filtered.csv')
response = requests.post(f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&rettype=fasta&retmode=text', {'id': ",".join(sequences_extended_filtered.index)})
with open('../datasets/sequences_filtered.fasta', 'w') as f:
    f.write(response.text)

In [9]:
!clustalo -i ../datasets/sequences_filtered.fasta --guidetree-out=tree --force

^C
